# 07 - Création d'un dataset de la fusion des datasets de clean metrics et clean jobs/job_events

Ce notebook génère 1 csv :

- dataset_for_preprocess.csv qui fusionne les données du dataset de metrics et jobs/job_events

Etapes : 

B) Import des datasets : 

- metrics est quasi-brut (la colonne events est fractionnée et tous les identifiants d'évènements sont des nombres)

- jobs (le dataset fusionné des données de jobs et job_events) dont chaque ligne est un job, un job est unique et il a un début et une fin

C) Réduction du dataset jobs et réductions succesives du dataset metrics (les lignes évènements avec certains identifiants)

D) Concaténation des datasets jobs et metrics (suppression des lignes inutiles : toutes celles de jobs et metrics si aucun jobId associé)


# A. Imports

## a) Librairies

In [475]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## b) Datasets

In [476]:
# source path to datasets
path = '../data/'
metrics = 'metrics/clean_merge_metrics_dataset.csv'
jobs = 'jobs/merge_raw_jobs_and_clean_jobevents_dataset.csv'

save_csv = '../data/dataset_for_preprocess_id_events_filtered_07.csv'

# B. Jeux de données

### a) Metrics

In [477]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(os.path.join(path, metrics), index_col=0)
print(f'metrics dataset shape {metrics_df.shape}')

metrics dataset shape (3510431, 14)


In [478]:
# la colonne timestamp contient-elle des valeurs en double ?
metrics_df['timestamp'].duplicated().any()

True

In [479]:
metrics_df['timestamp'].min()

'2022-04-15 05:55:06.678000+00:00'

In [480]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
groupes = metrics_df.groupby('timestamp').groups
print(len(groupes.keys()))

1242037


In [481]:
metrics_df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil L,iFoil,Gen. 2,Total Pages Counter,22881,Viktor,Operator,IDLE,36192,100000
1,iFoil,2022-04-15 06:06:56.278000+00:00,INFO,391,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,31092,Viktor,Operator,IDLE,36192,100000
2,PLC,2022-04-15 06:06:56.418000+00:00,INFO,330,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,1792992,Viktor,Operator,IDLE,36192,100000


### b) Jobs

In [482]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, jobs), index_col=0)
print(f'jobs dataset shape {jobs_df.shape}')

jobs dataset shape (16295, 42)


In [483]:
# la colonne started_at contient-elle des valeurs en double ?
jobs_df['started_at'].duplicated().any()

False

In [484]:
jobs_df['started_at'].min()

'2021-06-18 09:22:46.866000+00:00'

In [485]:
jobs_df.head(3)

,started_at,ended_at,paperHeight_job,paperWidth_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,jobId,total_copies_requested,LED,...,leftMargin_remoteScannerRegistration,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,total_copies,varnishConsumptionVarnish_3d_event
0,2021-06-18 09:22:46.866000+00:00,2021-06-18 09:22:46.866000+00:00,520,740,0,0,0.0,1624008166,3,10,...,0,1500,1500,1500,16,16,1,UNDEFINED,0,0.000000
1,2021-06-18 10:08:13.716000+00:00,2021-06-18 10:10:22.257000+00:00,740,520,0,0,0.0,1624010893,100,30,...,0,1500,1500,1500,16,10,1,ERROR,3,1.440239
2,2021-06-18 10:11:52.165000+00:00,2021-06-18 10:18:20.294000+00:00,740,520,0,0,0.0,1624011111,100,30,...,0,1500,1500,1500,16,10,1,CANCELED,70,33.607494


Les tailles des datasets sont déséquilibrés :

- 3510431 lignes pour metrics

- 16295 lignes pour jobs

Les dates de début sont différentes :

- '2022-04-15 05:55:06.678000+00:00' pour metrics

- '2021-06-18 09:22:46.866000+00:00' pour jobs

Le dataset metrics compte 1242037 doublons pour la colonne timestamp

# C. Equilibrage des jeux de données

## a) Réduction de jobs

In [486]:
# concordance des données de temps dans un même cadre
jobs_reduced = jobs_df[jobs_df.started_at > metrics_df.timestamp.min()]
jobs_reduced.reset_index()
jobs_reduced.shape

(10516, 42)

## b) Réduction de metrics

### Par importance d'identifiant d'évènement

In [487]:
# lignes avec des évènements identifiant une maintenance
id_to_drop_1 = [391, 330, 377, 407, 332, 331, 313, 333, 376, 372, 344, 343, 371, 358, 334, 311, 472, 408, 406, 350, 2, 352, 346]
# lignes avec des évènements identifiant une intervention humaine
id_to_drop_2 = [352, 324, 381, 440, 385, 405, 447, 388, 320, 417, 444, 329, 315, 384, 345, 349, 466, 419]
# fusion des listes d'identifiants à supprimer
id_to_drop = id_to_drop_1 + id_to_drop_2
# on supprime les lignes avec des évènements identifiant une maintenance
metrics_by_identification = metrics_df[~metrics_df['identification_events'].isin(id_to_drop)]
metrics_by_identification.shape

(3424346, 14)

In [488]:
# on vérifie la distribution par criticité
metrics_by_identification['identification_events'].value_counts()

0       3416284
454        2845
1000       1282
355         793
386         562
356         328
357         304
1003        294
325         287
445         240
354         195
480         160
453         155
387         154
1002        101
321          66
359          58
380          50
351          41
446          38
1001         22
323          15
479          11
430          11
460           9
411           9
471           8
475           5
416           4
389           3
476           3
1004          2
322           2
418           2
326           2
327           1
Name: identification_events, dtype: int64

### Par nombre d'identifiant d'évènement

In [489]:
# Calculer le nombre de valeurs d'identification_events pour chaque groupe
counts = metrics_by_identification.groupby('identification_events')['identification_events'].transform('count')
# Filtrer les lignes où le nombre de valeurs d'identification_events est inférieur à 20
metrics_reduced = metrics_by_identification[counts >= 50]

In [490]:
# on vérifie la distribution par identification_events
metrics_reduced['identification_events'].value_counts()

0       3416284
454        2845
1000       1282
355         793
386         562
356         328
357         304
1003        294
325         287
445         240
354         195
480         160
453         155
387         154
1002        101
321          66
359          58
380          50
Name: identification_events, dtype: int64

### Premier équilibrage par identifiant d'évènement

In [491]:
# fonction permettant de reduire le nombre de ligne d'une classe à n lignes
def balance_dataframe_by_identification(df, identification, n):
    # on filtre les lignes du df qui ont la valeur criticality dans la colonne 'criticality_events'
    identification_df = df[df['identification_events'] == identification]
    # on selectionne un nombre aléatoire de lignes
    identification_df_sample = identification_df.sample(n=n)
    # on stoke les index des lignes du df qui ne sont pas dans le sample
    index_to_delete = identification_df[~identification_df.index.isin(identification_df_sample.index)].index
    # suppression des lignes exclu du sample
    return df.drop(index_to_delete)

In [492]:
# on réduit le nombre de classe pour l'id 0
metrics_reduced = balance_dataframe_by_identification(metrics_reduced,0,2845)
# on vérifie la distribution par identification_events
metrics_reduced['identification_events'].value_counts()

454     2845
0       2845
1000    1282
355      793
386      562
356      328
357      304
1003     294
325      287
445      240
354      195
480      160
453      155
387      154
1002     101
321       66
359       58
380       50
Name: identification_events, dtype: int64

In [493]:
# on vérifie la distribution par criticité
metrics_reduced['criticality_events'].value_counts()

ERROR      5764
INFO       1514
WARNING     596
Name: criticality_events, dtype: int64

# D. Création du datatset pour le pre-processing

In [494]:
metrics_reduced['jobId'] = 0
metrics_reduced.shape

(10719, 15)

## a) Analyse des dates

In [495]:
# # Vérifier si des dates communes existent
# dates_communes_exist = jobs_reduced['started_at'].isin(metrics_reduced['timestamp']).any()

# # Afficher le résultat
# if dates_communes_exist:
#     # Filtrer les dates communes
#     dates_communes = jobs_df['started_at'][jobs_df['started_at'].isin(metrics_df['timestamp'])]
    
#     # Compter le nombre de dates communes
#     nombre_dates_communes = len(dates_communes)
    
#     print(f"Des dates communes existent entre les ensembles de données: {nombre_dates_communes}")
# else:
#     print("Aucune date commune n'a été trouvée entre les ensembles de données.")


In [496]:
# création de dictionnaire du nombre de lignes par mois
def compare_datetime_series_shapes(s1, s2):
    s1_serie = {}
    s2_serie = {}
    s1 = pd.to_datetime(s1)
    s2 = pd.to_datetime(s2)
    for i in range(1,13):
        s1_month = s1[s1.dt.month == i]
        s2_month = s2[s2.dt.month == i]
        print(i, s1_month.shape, s2_month.shape)
        s1_serie[i] = s1_month.index
        s2_serie[i] = s2_month.index
    return s1_serie, s2_serie

In [497]:
# on liste les index des lignes par mois
#jobs_indexes_by_month, metrics_indexes_by_month = compare_datetime_series_shapes(jobs_reduced['started_at'], metrics_reduced['timestamp'])

In [498]:
# nombre de lignes par mois
# month = 5

# Selection des lignes de jobs en Mai
# jobs_df['started_at'] = pd.to_datetime(jobs_df['started_at'])
# jobs_batch = jobs_df[ (jobs_df['started_at'].dt.month == month)]
# jobs_batch = jobs_df.loc[jobs_df.index.isin(jobs_indexes_by_month.get(month))]
# jobs_batch.shape

# Selection des lignes de metrics en Mai
# metrics_df['timestamp'] = pd.to_datetime(metrics_df['timestamp'])
# metrics_batch = metrics_df[ (metrics_df['timestamp'].dt.month == month)]
# metrics_batch = metrics_df.loc[metrics_df.index.isin(metrics_indexes_by_month.get(month))]
# metrics_batch.shape

## b) Correspondance de dates entre metrics et jobs

### Pour chaque ligne de metrics quel est le jobId correspondant

In [499]:
import warnings
warnings.simplefilter("ignore")

In [500]:
# DataFrame 1 
# avec les intervalles de temps
dataframe1 = jobs_reduced[['started_at', 'ended_at', 'jobId']]
# conversion des colonnes au format datetime
cols_to_convert = ['started_at', 'ended_at']
dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)
dataframe1 = dataframe1.sort_values('started_at')
dataframe1 = dataframe1.reset_index()
dataframe1.head(3)

,index,started_at,ended_at,jobId
0,5779,2022-04-15 06:07:27.953000+00:00,2022-04-15 06:10:19.842000+00:00,1650002847
1,5780,2022-04-15 07:02:47.972000+00:00,2022-04-15 07:05:20.268000+00:00,1650006167
2,5781,2022-04-15 07:07:26.219000+00:00,2022-04-15 07:08:10.478000+00:00,1650006446


In [501]:
# DataFrame 2 
# avec les valeurs de date
dataframe2 = pd.DataFrame({
    'timestamp': metrics_reduced['timestamp'].values,
    'criticality_events': metrics_reduced['criticality_events'].values,
    'identification_events': metrics_reduced['identification_events'].values,
    'jobId': 0
}, index=None)
# conversion des colonnes au format datetime
dataframe2['timestamp'] = pd.to_datetime(dataframe2['timestamp'])
dataframe2 = dataframe2.sort_values('timestamp')
dataframe2 = dataframe2.reset_index()
dataframe2.head(3)

,index,timestamp,criticality_events,identification_events,jobId
0,4268,2022-04-15 05:58:14.754000+00:00,NaN,0,0
1,4269,2022-04-15 06:10:23.565000+00:00,ERROR,1000,0
2,4270,2022-04-15 06:20:57.022000+00:00,NaN,0,0


In [502]:
import time
# début du temps d'execution de la cellule
start = time.time()

# Parcours chaque ligne de job
for index1, row1 in dataframe1.iterrows():
    # creation des variables
    job_starts = row1.started_at.tz_localize(None)
    job_ends = row1.ended_at.tz_localize(None)

    # mois de référence
    month = job_starts.month
    if month != job_ends.month :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) | (dataframe2['timestamp'].dt.month == job_ends.month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')
    else :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')

    for index2, row2 in dataframe2_by_month.iterrows():
        # creation des variables
        metrics_timestamp = row2['timestamp'].tz_localize(None)
        # si le timestamp de la lignes de row2 de metrics est compris dans l'intervalle de temps du job de row1
        if metrics_timestamp >= job_starts :
            if metrics_timestamp <= job_ends :
                dataframe2.loc[index2, 'jobId'] = dataframe1.loc[index1, 'jobId']
    # # avant de passer à la row1 suivante
    # # on affiche l'index1 toutes les 1000 lignes et son temps d'execution
    print(f'[ job index {index1} = time {time.time() - start}]') if index1 % 1000 == 0 and index1 != 0 else None


[ job index 1000 = time 16.661453008651733]
[ job index 2000 = time 40.68084526062012]
[ job index 3000 = time 69.38685989379883]
[ job index 4000 = time 96.23442459106445]
[ job index 5000 = time 121.95805358886719]
[ job index 6000 = time 136.78272414207458]
[ job index 7000 = time 149.66488313674927]
[ job index 8000 = time 166.97123003005981]
[ job index 9000 = time 185.33247828483582]
[ job index 10000 = time 203.78303408622742]


In [503]:
# on vérifie la distribution par identification_events pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
dataframe2[dataframe2['jobId'] != 0]['identification_events'].value_counts()

0       988
454     668
325     228
355     166
357      83
356      69
387      68
445      50
354      50
453      42
1003     41
359      26
386      25
1002     19
480      12
1000      5
321       3
Name: identification_events, dtype: int64

In [511]:
print(f'Nombre de ligne de metrics : {dataframe2.shape[0]} (dont {dataframe2[dataframe2.jobId != 0].shape[0]} avec un jobId associé)')

Nombre de ligne de metrics : 10719 (dont 2543 avec un jobId associé)


## c) Fusion des dataframes jobs et metrics

In [512]:
merged_df = jobs_reduced.merge(dataframe2, on='jobId')
merged_df.shape

(2543, 46)

In [513]:
# on vérifie la distribution par criticité
merged_df['identification_events'].value_counts()

0       988
454     668
325     228
355     166
357      83
356      69
387      68
445      50
354      50
453      42
1003     41
359      26
386      25
1002     19
480      12
1000      5
321       3
Name: identification_events, dtype: int64

In [516]:
# on vérifie la distribution par criticité
merged_df['criticality_events'].value_counts()

ERROR      1529
INFO         14
WARNING      12
Name: criticality_events, dtype: int64

### Dataframe ne contenant que des criticality = ERROR

In [517]:
merged_df_error = merged_df[merged_df.criticality_events == 'ERROR']

In [518]:
# on vérifie la distribution par criticité
merged_df_error['identification_events'].value_counts()

454     668
325     228
355     148
357      77
387      68
356      67
445      50
354      50
453      42
1003     41
359      26
386      25
1002     19
480      12
1000      5
321       3
Name: identification_events, dtype: int64

### Equilibrage des classes identification_events en fonction de l'id le plus fréquent des lignes ERROR

In [522]:
# on identifie la classe identification_events la plus fréquente dans les ERROR
identification_counts = merged_df[merged_df['criticality_events'] == 'ERROR']['identification_events'].value_counts()
most_frequent_identification_event = identification_counts.idxmax()
nombre_lignes = identification_counts[most_frequent_identification_event]
print(f"L'id '{most_frequent_identification_event}' est le plus fréquent des lignes ERROR avec {nombre_lignes} lignes.")

L'id '454' est le plus fréquent des lignes ERROR avec 668 lignes.


In [ ]:
# TODO balance_dataframe_by_identification
# itérer dans le dataframe pour soit 
# réduire les classes trop représentées
# soit augmenter les moins représentées 
# par rapport à un seuil n


## d) Output

In [515]:
merged_df.to_csv(path_or_buf=save_csv)